<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/reasoning_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reasoning Agent
Idea: is generate a plan with `o1-mini` and then execute each step with `gpt-4o-mini`.
Implementation: Planning agent is completed; implementation agent - work in progress

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Import OpenAI key
from google.colab import userdata
openai_api_key = userdata.get('OpenAI')

In [4]:
import copy
import json
from openai import OpenAI

from utils import o1_tools

client = OpenAI(api_key=openai_api_key)
O1_MODEL = 'o1-mini'
GPT_MODEL = 'gpt-4o-mini'

## Definition

In [8]:
# Initialize the message list
message_list = []

# Define the initial context for the application
context = {
    'Providers': {
        'Caregivers': 5  # Number of cargivers
    },
    'care_requests': [
        {
            'Locations': ['Queens','Brooklyn','Namhattan'],
            'requests': [
                {'patient': 'patient1', 'location': 'Queens', 'day': 'Monday', 'time': '10am-12pm'},
                {'patient': 'patient2', 'location': 'Queens', 'day': 'Monday', 'time': '1pm-3pm'},
                {'patient': 'patient3', 'location': 'Brooklyn', 'day': 'Tuesday', 'time': '10am-12pm'},
                {'patient': 'patient4', 'location': 'Manhattan', 'day': 'Wednesday', 'time': '11am-2pm'},
                {'patient': 'patient5', 'location': 'Queens', 'day': 'Wednesday', 'time': '3pm-5pm'},
                {'patient': 'patient6', 'location': 'Brroklyn', 'day': 'Thursday', 'time': '1am-5pm'},
                {'patient': 'patient7', 'location': 'Queens', 'day': 'Thursday', 'time': '10am-12pm'},
                {'patient': 'patient8', 'location': 'Queens', 'day': 'Friday', 'time': '10am-12pm'}
                ] # Dimensions in cm
        }
    ],
    'available_caregivers': ['CG1', 'CG2', 'CG3', 'CG4', 'CG5'],
    'suppliers': {
        'CG1': {
            'availability': {
                'Location': ['Queens','Manhattan'],
                'working_days':['Monday','Tuesday','Wednesday'],
                'start_time': '09:00',
                'end_time': '17:00'
                }
            },
         'CG2': {
            'availability': {
                'Location': ['Queens'],
                'working_days':['Monday','Thursday','Friday'],
                'start_time': '09:00',
                'end_time': '17:00'
            },
         },
        'CG3': {
            'availability': {
                'Location': ['Brooklyn'],
                'working_days':['Monday','Tuesday','Thursday','Friday'],
                'start_time': '09:00',
                'end_time': '17:00'
            }
        },
        'CG4': {
            'availability': {
                'Location': ['Broklyn', 'Manhattan'],
                'working_days':['Monday','Tuesday','Wednesday','Thursday','Friday'],
                'start_time': '09:00',
                'end_time': '17:00'
            }
        },
        'CG5': {
            'availability': {
                'Location': ['Manhattan', 'Brooklyn'],
                'start_time': '09:00',
                'end_time': '17:00'
            }
        }

    }
}

# Store the initial state of context
initial_context = copy.deepcopy(context)

In [9]:
# Prompt for the planning model
o1_prompt = """
You are a scheduling assistant. The first input you will receive will be a complex task that needs to be carefully reasoned through to solve.
Your task is to review the challenge, and create a detailed plan to optimally match service providers (caregivers) with service users (patients)
while considering cargivers work days and work hours, avoiding double-booking and taking into account the travel travel time between the service location.
You may suggest adding new caregivers working hours or locations if you are not able to cover all the sevice requests.

You will have access to an LLM agent that is responsible for executing the plan that you create and will return results.

The LLM agent has access to the following functions:
    - get_address(ID)
        - This gets the addresses for service provider and service users
    - calculate_travel_time(origin_address, destination_address)
        - This function calculate travel time between locations
    -add_caregiver(service_location)
        - This function adds cargivers to udnerserved location


When creating a plan for the LLM to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - **Main actions are numbered** (e.g., 1, 2, 3).
    - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
        - **Sub-actions should start on new lines**
    - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
    - **For actions that require using one of the above functions defined**, write a step to call a function using backticks for the function name (e.g., `call the get_inventory_status function`).
        - Ensure that the proper input arguments are given to the model for instruction. There should not be any ambiguity in the inputs.
    - **The last step** in the instructions should always be calling the `instructions_complete` function. This is necessary so we know the LLM has completed all of the instructions you have given it.
    - **Detailed steps** The plan generated must be extremely detailed and thorough with explanations at every step.
Use markdown format when generating the plan with each step and sub-step.

Please find the scenario below.
"""

In [10]:
# System prompt for the execution model
gpt4o_system_prompt = """
You are a helpful assistant responsible for executing the policy on handling incoming orders. Your task is to follow the policy exactly as it is written and perform the necessary actions.

You must explain your decision-making process across various steps.

# Steps

1. **Read and Understand Policy**: Carefully read and fully understand the given policy on handling incoming orders.
2. **Identify the exact step in the policy**: Determine which step in the policy you are at, and execute the instructions according to the policy.
3. **Decision Making**: Briefly explain your actions and why you are performing them.
4. **Action Execution**: Perform the actions required by calling any relevant functions and input parameters.

POLICY:
{policy}

"""


#### Describe functions that will be passed to the 4o-mini helper

In [11]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_address",
            "description": "Retrives address of caregivers and patients",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The unique identifier for the caregiver or patient."
                    },
                },
                "required": ["id"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_travel_time",
            "description": "Cacculates travel time between locations",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin_address": {
                        "type": "string",
                        "description": "The location were trip starts.",
                    },
                    "destination_addres": {
                        "type": "string",
                        "description": "The location were trip ends",
                    },
                },
                "required": ["origina_address", "destination_address"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "add_caregiver",
            "description": "Adds caregivers for the underserved locations",
            "parameters": {
                "type": "object",
                "properties": {
                    "service_location": {
                        "type": "string",
                        "description": "The location were more caregiers need to be added.",
                    },
                },
                "required": "service_location",
                "additionalProperties": False,
            }
        },
    }
]

#### These are the instantiations of the functions the 4o-mini helper will use.

In [12]:
# Function Definitions
def get_address(ID):
    pass
    return 'address'

def calculate_travel_time(product_id):
    time = random.randint(1, 100)
    return time

def add_caregiver(location):
    id = random.randint(1, 100)
    caregiver = {
        'id': id,
        'location': location
    }
    return caregiver

#### Knit together the process. 1) call o1 to generate a plan, 2) call 4o-mini to execute the plan

In [18]:
def process_scenario(scenario):
    append_message({'type': 'status', 'message': 'Generating plan...'})

    plan = call_o1(scenario)

    messages = append_message({'type': 'plan', 'content': plan})

    #append_message({'type': 'status', 'message': 'Executing plan...'})

    #messages = call_gpt4o(plan)

    #append_message({'type': 'status', 'message': 'Processing complete.'})

    return messages

##### Helper function

In [14]:
def append_message(message):
    message_list.append(message)
    # Optionally, print the message for immediate feedback
    message_type = message.get('type', '')
    if message_type == 'status':
        print(message['message'])
    elif message_type == 'plan':
        print("\nPlan:\n", message['content'])
    elif message_type == 'assistant':
        print("\nAssistant:\n", message['content'])
    elif message_type == 'function_call':
        print(f"\nFunction call: {message['function_name']} with arguments {message['arguments']}")
    elif message_type == 'function_response':
        print(f"\nFunction response for {message['function_name']}: {message['response']}")
    else:
        # Handle any other message types or default case
        print(message.get('content', ''))

#### Calls the planner, o1 model. The response will be the plan that will be provided to the  4o-mini helper.

In [15]:
def call_o1(scenario):
    prompt = f"""
{o1_prompt}

Scenario:
{scenario}

Please provide the next steps in your plan."""

    response = client.chat.completions.create(
        model=O1_MODEL,
        messages=[{'role': 'user', 'content': prompt}]
    )
    plan = response.choices[0].message.content

    return plan

#### Call 4o-mini to execute the plan. This will loop until the plan is complete

In [ ]:
def call_gpt4o(plan):
    gpt4o_policy_prompt = gpt4o_system_prompt.replace("{policy}", plan)
    messages = [
        {'role': 'system', 'content': gpt4o_policy_prompt},
    ]

    while True:
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            tools=TOOLS,
            parallel_tool_calls=False
        )

        assistant_message = response.choices[0].message.to_dict()
        print(assistant_message)
        messages.append(assistant_message)

        append_message({'type': 'assistant', 'content': assistant_message.get('content', '')})

        if (response.choices[0].message.tool_calls and
            response.choices[0].message.tool_calls[0].function.name == 'instructions_complete'):
            break

        if not response.choices[0].message.tool_calls:
            continue

        for tool in response.choices[0].message.tool_calls:
            tool_id = tool.id
            function_name = tool.function.name
            input_arguments_str = tool.function.arguments

            append_message({'type': 'tool_call', 'function_name': function_name, 'arguments': input_arguments_str})

            try:
                input_arguments = json.loads(input_arguments_str)
            except (ValueError, json.JSONDecodeError):
                continue

            if function_name in function_mapping:
                try:
                    function_response = function_mapping[function_name](**input_arguments)
                except Exception as e:
                    function_response = {'error': str(e)}
            else:
                function_response = {'error': f"Function '{function_name}' not implemented."}

            try:
                serialized_output = json.dumps(function_response)
            except (TypeError, ValueError):
                serialized_output = str(function_response)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_id,
                "content": serialized_output
            })

            append_message({'type': 'tool_response', 'function_name': function_name, 'response': serialized_output})

    return messages

## Execution

In [19]:
# Example usage
scenario_text = ("We need to create a caregiver schedule for a week. "
                 "Please generate a plan that gets caregivers availability "
                 "and matches it with the service requests.\n\n"
                 "The plan should consider patient locations"
                 "and match it with the caregiver working location."
                 "The should avoid doublebooking caregivers and"
                 "and account for travel time between work location."
                 "the plan may include a suggestion to add cargivers"
                 "covering underserved locations")

# Process the scenario
messages = process_scenario(scenario_text)

Generating plan...

Plan:
 ```markdown
# Caregiver Scheduling Plan

1. **Retrieve Service Requests for the Week**
    - 1a. Compile a list of all service requests for the upcoming week, including patient IDs, requested service times, and service locations.
    - 1b. For each patient ID in the service requests, `call the get_address` with the patient ID to obtain their address.

2. **Gather Caregiver Availability and Locations**
    - 2a. Compile a list of all caregivers, including their available work days, work hours, and current work locations.
    - 2b. For each caregiver, `call the get_address` with the caregiver ID to obtain their work address.

3. **Match Service Requests with Caregiver Availability**
    - 3a. For each service request:
        - 3a1. Identify caregivers available on the requested service day.
        - 3a2. From the available caregivers, filter those whose work hours encompass the requested service time.
    - 3b. For each eligible caregiver:
        - 3b1. `cal

#### Print messages
<span style="color:green;">Note: Your results may differ from those in the video as the models outputs may change with each run.</span>


In [23]:
#for x in messages:
    #print(x)
    #print('\n\n')